In [ ]:
#!pip install python-slugify universal_pathlib

In [57]:
import time
import datetime
import urllib.request

import pandas as pd
from slugify import slugify
from upath import UPath as Path

In [80]:
files_dir = Path("/Users/ewheeler/dev/UNDatathon/data/gdelt_files/")

In [ ]:
territory_identifiers = pd.read_csv(files_dir / "country-codes/country_codes.csv")

In [44]:
df_fips = territory_identifiers[['Country (geonames)', 'fips (geonames)']].dropna()
df_fips = df_fips.rename(columns={'Country (geonames)': 'name', 'fips (geonames)': 'fips'})
df_fips

,name,fips
0,Algeria,AG
1,Egypt,EG
2,Libya,LY
3,Morocco,MO
4,Sudan,SU
...,...,...
278,Tokelau,TL
279,Tonga,TN
280,Tuvalu,TV
281,Wallis and Futuna,WF


In [47]:
countries = list(df_fips.fips.values)

In [49]:
countries[:5]

['AG', 'EG', 'LY', 'MO', 'SU']

In [7]:
#df = pd.read_csv("/Users/ewheeler/Downloads/results-20231104074458.csv")
#df = pd.read_csv("/Users/ewheeler/Downloads/results-20231104074641.csv", on_bad_lines='skip')
#df

In [92]:
#resp = requests.get("https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:UP&mode=timelinevol&timezoom=yes")

In [ ]:
example_highcharts_export_redirect_url = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:KE&mode=timelinevol&timezoom=yes"
example_article_list_csv_url = "https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:AL&mode=artlist&maxrecords=75&format=csv&sort=hybridrel"

In [ ]:
%%time
query = "commodity"
modes = ["timelinevolraw", "timelinetone", "timelinevol"]
for fips in countries[3:]:
    for mode in modes:
        source = f"https://api.gdeltproject.org/api/v2/doc/doc?format=csv&startdatetime=20200101000000&enddatetime=20231031235959&query={query.replace(' ', '%20')}%20sourcecountry:{fips}&mode={mode}&timezoom=yes"
        destination = files_dir / f"{query}/{fips}_{slugify(query)}_{mode}.csv"
        if not destination.exists():
            local_filename, headers = urllib.request.urlretrieve(source, destination)
            print(local_filename)
            # endpoint is rate limited.
            # error msg says to keep big queries to
            # one per 5s, so sleep after each call
            time.sleep(6)
    source = f"https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime=20200101000000&enddatetime=20231031235959&query={query.replace(' ', '%20')}%20sourcecountry:{fips}&mode=artlist&maxrecords=75&format=csv&sort=hybridrel"
    destination = files_dir / f"{query}/{fips}_{slugify(query)}_artlist.csv"
    if not destination.exists():
        local_filename, headers = urllib.request.urlretrieve(source, destination)
        print(local_filename)
        # endpoint is rate limited.
        # error msg says to keep big queries to
        # one per 5s, so sleep after each call
        time.sleep(6)

/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/MO_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/SU_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/TS_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/WI_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/IO_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/BY_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/CN_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/DJ_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/ER_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/ET_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_files/commodity/FS_commodity_timelinevol.csv
/Users/ewheeler/dev/UNDatathon/data/gdelt_f

In [117]:
dfs = {'artlist': list(), 'timelinetone': list(),
       'timelinevolraw': list(), 'timelinevol': list()}
for f in Path(files_dir / "commodity").glob("*.csv"):
    fips, query, api_mode = f.stem.split('_')
    # XXX change for desired query
    if query == 'commodity':
        try:
            df = pd.read_csv(f)
        except pd.errors.EmptyDataError as e:
            print(f.stem, e)
            continue
        df['fips'] = fips
        df['query'] = query
        df['api_mode'] = api_mode
        dfs_list = dfs[api_mode]
        dfs_list.append(df)
        dfs.update({api_mode: dfs_list})

CQ_commodity_timelinevolraw No columns to parse from file
GQ_commodity_artlist No columns to parse from file
DJ_commodity_artlist No columns to parse from file
KR_commodity_timelinevolraw No columns to parse from file
TK_commodity_artlist No columns to parse from file
AN_commodity_artlist No columns to parse from file
SC_commodity_artlist No columns to parse from file
EK_commodity_timelinevolraw No columns to parse from file
WI_commodity_artlist No columns to parse from file
TL_commodity_timelinevolraw No columns to parse from file
TB_commodity_timelinevolraw No columns to parse from file
GL_commodity_timelinetone No columns to parse from file
NS_commodity_artlist No columns to parse from file
MH_commodity_artlist No columns to parse from file
CQ_commodity_timelinetone No columns to parse from file
RM_commodity_artlist No columns to parse from file
MB_commodity_artlist No columns to parse from file
CQ_commodity_artlist No columns to parse from file
VT_commodity_artlist No columns to pa

In [119]:
for api_mode, dfs_list in dfs.items():
    if len(dfs_list) == 0:
        continue
    df = pd.concat(dfs_list)
    try:
        df = df.drop(columns=['Invalid/Unsupported Country.'])
    except KeyError:
        pass
    df['api_mode'] = api_mode
    # XXX change for query used in previous cell!
    df.to_feather(files_dir / f'gdelt_commodity_{api_mode}.feather')

/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [118]:
"""
import os

for f in files_dir.glob('*.csv'):
    print(f)
    try:
        fips, query, api_mode, dt = f.stem.split('_')
    except ValueError:
        continue
    new_stem = "_".join([fips, query, api_mode])
    p = Path(str(f).replace(f.stem, new_stem))
    print(p)
    os.rename(f, p)
"""

'\nimport os\n\nfor f in files_dir.glob(\'*.csv\'):\n    print(f)\n    try:\n        fips, query, api_mode, dt = f.stem.split(\'_\')\n    except ValueError:\n        continue\n    new_stem = "_".join([fips, query, api_mode])\n    p = Path(str(f).replace(f.stem, new_stem))\n    print(p)\n    os.rename(f, p)\n'